In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path

from src.processing.splitting import time_based_split
from src.processing.scaling import scale_features
from src.processing.sequence_creator import create_sequences
from src.training.lstm_training import train_lstm
from config import config


df_btc = pd.read_csv(
    config.DATA_DIR / "processed" / "crypto_prices" / "btc.csv",
    parse_dates=["date"],
    index_col="date",
)
df_eth = pd.read_csv(
    config.DATA_DIR / "processed" / "crypto_prices" / "eth.csv",
    parse_dates=["date"],
    index_col="date",
)

## Split


In [2]:
X_train_btc, y_train_btc, X_val_btc, y_val_btc, X_test_btc, y_test_btc = (
    time_based_split(
        df_btc,
        targets=["logPriceChange"],
        test_months=12,
        val_months=3,
        lags=30,
    )
)

X_train_eth, y_train_eth, X_val_eth, y_val_eth, X_test_eth, y_test_eth = (
    time_based_split(
        df_eth,
        targets=["logPriceChange"],
        test_months=12,
        val_months=3,
        lags=30,
    )
)

/Users/matti/coding/cryptoCapstone/src/processing/splitting.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[target] = train[target].shift(-1)
/Users/matti/coding/cryptoCapstone/src/processing/splitting.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val[target] = val[target].shift(-1)
/Users/matti/coding/cryptoCapstone/src/processing/splitting.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

In [3]:
print(y_test_btc.tail())
print(y_test_btc.head())

            logPriceChange
date                      
2022-12-15       -0.042190
2022-12-16        0.008828
2022-12-17       -0.002212
2022-12-18       -0.019176
2022-12-19        0.027989
            logPriceChange
date                      
2021-11-21       -0.042455
2021-11-22        0.022481
2021-11-23       -0.022639
2021-11-24        0.017512
2021-11-25       -0.066874


## Scale


In [4]:
X_train_scaled_btc, X_val_scaled_btc, X_test_scaled_btc, scaler_btc = scale_features(
    X_train_btc, X_val_btc, X_test_btc
)

X_train_scaled_eth, X_val_scaled_eth, X_test_scaled_eth, scaler_eth = scale_features(
    X_train_eth, X_val_eth, X_test_eth
)

## Sequence Creation


In [5]:
X_train_seq_btc, y_train_seq_btc = create_sequences(
    X_train_scaled_btc, y_train_btc, seq_length=30
)
X_val_seq_btc, y_val_seq_btc = create_sequences(
    X_val_scaled_btc, y_val_btc, seq_length=30
)
X_test_seq_btc, y_test_seq_btc = create_sequences(
    X_test_scaled_btc, y_test_btc, seq_length=30
)

print(f"X_train_seq_btc shape: {X_train_seq_btc.shape}")
print(f"y_train_seq_btc shape: {y_train_seq_btc.shape}")

X_train_seq_eth, y_train_seq_eth = create_sequences(
    X_train_scaled_eth, y_train_eth, seq_length=30
)
X_val_seq_eth, y_val_seq_eth = create_sequences(
    X_val_scaled_eth, y_val_eth, seq_length=30
)
X_test_seq_eth, y_test_seq_eth = create_sequences(
    X_test_scaled_eth, y_test_eth, seq_length=30
)

X_train_seq_btc shape: (1662, 30, 72)
y_train_seq_btc shape: (1662, 1)


# BTC


### Model Tuning

In [6]:
from src.training.hyperparameters_tuning import tune_hyperparameters

study_lstm_btc = tune_hyperparameters(
    X_train_seq_btc,
    y_train_seq_btc,
    X_val_seq_btc,
    y_val_seq_btc,
    train_lstm,
    n_trials=50,
)

[I 2025-02-10 15:36:44,608] A new study created in memory with name: no-name-81e352fc-3b56-4ed0-a768-c465da172968


Epoch 1/100
 51/104 ━━━━━━━━━━━━━━━━━━━━ 4s 79ms/step - loss: 1.2406 - mae: 0.7052

## Model Training

In [ ]:
# best parameters from the Optuna study
best_params_btc = study_lstm_btc.best_params_btc
# best_params_btc = {
#     "num_lstm_layers": 3,
#     "lstm_units_1": 256,
#     "lstm_units_2": 32,
#     "lstm_units_3": 64,
#     "num_dense_layers": 1,
#     "dense_units_1": 224,
#     "dense_units_2": 224,
#     "dense_units_3": 64,
#     "dropout_rate": 0.30000000000000004,
#     "learning_rate": 0.00035070707045515984,
#     "batch_size": 64,
# }


# Format the parameters correctly
formatted_params = {
    "lstm_units": [
        best_params_btc[f"lstm_units_{i+1}"] for i in range(best_params_btc["num_lstm_layers"])
    ],
    "dense_units": [
        best_params_btc[f"dense_units_{i+1}"]
        for i in range(best_params_btc["num_dense_layers"])
    ],
    "dropout_rate": best_params_btc["dropout_rate"],
    "learning_rate": best_params_btc["learning_rate"],
    "batch_size": best_params_btc["batch_size"],
    "epochs": 300,
}

# Train the model with the formatted parameters
model_btc, history_btc = train_lstm(
    X_train_seq_btc,
    y_train_seq_btc,
    X_val_seq_btc,
    y_val_seq_btc,
    params=formatted_params,
)

NameError: name 'study_lstm_btc' is not defined

In [ ]:
# save weights
model_btc.save_weights(
    config.DATA_DIR
    / "weights"
    / "log_price_change"
    / "log_price_change_weights_btc.weights.h5"
)

#  generate predictions
y_pred_btc = model_btc.predict(X_test_seq_btc).flatten()

# create new df and save it to csv
predictions_btc = pd.DataFrame(
    {
        "date": y_test_btc.index[30:],
        "predictedLogPriceChange": y_pred_btc,
    }
)


predictions_btc.to_csv(
    config.DATA_DIR / "predictions" / "log_price_change_predictions_btc.csv",
    index=False,
)

12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step


# ETH

### Model Tuning

In [ ]:
study_lstm_eth = tune_hyperparameters(
    X_train_seq_eth,
    y_train_seq_eth,
    X_val_seq_eth,
    y_val_seq_eth,
    train_lstm,
    n_trials=50,
)

## Model Training

In [ ]:
# best parameters from the Optuna study
best_params_eth = study_lstm_eth.best_params_eth
# best_params_eth = {
#     "num_lstm_layers": 2,
#     "lstm_units_1": 160,
#     "lstm_units_2": 32,
#     "lstm_units_3": 32,
#     "num_dense_layers": 1,
#     "dense_units_1": 96,
#     "dense_units_2": 64,
#     "dense_units_3": 192,
#     "dropout_rate": 0.2,
#     "learning_rate": 0.0005577190382479473,
#     "batch_size": 32,
# }


# Format the parameters correctly
formatted_params = {
    "lstm_units": [
        best_params_eth[f"lstm_units_{i+1}"] for i in range(best_params_eth["num_lstm_layers"])
    ],
    "dense_units": [
        best_params_eth[f"dense_units_{i+1}"]
        for i in range(best_params_eth["num_dense_layers"])
    ],
    "dropout_rate": best_params_eth["dropout_rate"],
    "learning_rate": best_params_eth["learning_rate"],
    "batch_size": best_params_eth["batch_size"],
    "epochs": 300,
}

# Train the model with the formatted parameters
model_eth, history_eth = train_lstm(
    X_train_seq_eth,
    y_train_seq_eth,
    X_val_seq_eth,
    y_val_seq_eth,
    params=formatted_params,
)

Epoch 1/300
52/52 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step - loss: 0.0194 - mae: 0.1052 - val_loss: 0.0025 - val_mae: 0.0384
Epoch 2/300
52/52 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - loss: 0.0085 - mae: 0.0691 - val_loss: 0.0031 - val_mae: 0.0460
Epoch 3/300
52/52 ━━━━━━━━━━━━━━━━━━━━ 2s 42ms/step - loss: 0.0067 - mae: 0.0609 - val_loss: 0.0022 - val_mae: 0.0372
Epoch 4/300
52/52 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - loss: 0.0054 - mae: 0.0528 - val_loss: 0.0015 - val_mae: 0.0300
Epoch 5/300
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.0049 - mae: 0.0504 - val_loss: 0.0017 - val_mae: 0.0335
Epoch 6/300
52/52 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - loss: 0.0046 - mae: 0.0474 - val_loss: 0.0020 - val_mae: 0.0355
Epoch 7/300
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 0.0046 - mae: 0.0474 - val_loss: 0.0017 - val_mae: 0.0331
Epoch 8/300
52/52 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - loss: 0.0046 - mae: 0.0471 - val_loss: 0.0016 - val_mae: 0.0317
Epoch 9/300
52/52 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - lo

In [ ]:
# save weights
model_eth.save_weights(
    config.DATA_DIR
    / "weights"
    / "log_price_change"
    / "log_price_change_weights_eth.weights.h5"
)

#  generate predictions
y_pred_eth = model_eth.predict(X_test_seq_eth).flatten()

# create new df and save it to csv
predictions_eth = pd.DataFrame(
    {
        "date": y_test_eth.index[30:],
        "predictedLogPriceChange": y_pred_eth,
    }
)


predictions_eth.to_csv(
    config.DATA_DIR / "predictions" / "log_price_change_predictions_eth.csv",
    index=False,
)

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
